In [4]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime as dt
from datetime import date
import calendar
from calendar import weekday, day_name
import re

In [5]:
key = pd.read_csv('key.csv')
weather = pd.read_csv('weather.csv', parse_dates=['date'])
train = pd.read_csv('train.csv', parse_dates=['date'])

In [6]:
train = pd.merge(train,key)

In [7]:
rt = train[train['units'] > 0].sort_values(by=['units'], ascending=False).reset_index(drop=True)

In [8]:
token = ('+FCFCTSGRRADZSNSGGSPLICFG+FGBRUPHZFUVADUDSPOSOSASSPYSQDRSHFZMIPRBCBLVC')

In [9]:
# codesum 분류를 위한 기준 설정 및 추출 함수
ls = np.array(['+FC', 'FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG+', 
               'FG', 'BR', 'UP', 'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SO', 'SA', 'SS',
                'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC'])



def validation(token):
    result3 = []
    for each in ls:
        if each in token:
            token = token.replace(each,'')
            result3.append(each)
    return result3


def extract_cs():
    test = list(weather.codesum.unique())
    result = []
    for each in test:
        result.append(validation(each))
    return result


In [10]:
# 위 codesum 추출 함수를 기반으로 codesum value에 해당하는 컬럼 및 값 부여
cs_iter = (set(validation(token)) for x in weather.codesum)
cs = sorted(set.union(*cs_iter))

dummies = pd.DataFrame(np.zeros((len(weather), len(cs))), columns = cs)


for i, gen in enumerate(weather.codesum):
    dummies.ix[i, validation(gen)] = '1'
    
weather_cs = weather.join(dummies)

/Users/woongkook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [11]:
weather_cs.DZ.value_counts()

0.0    20020
1        497
Name: DZ, dtype: int64

In [12]:
df = pd.merge(rt, weather_cs, on=['date', 'station_nbr'])

In [13]:
def get_holidays(fpath):
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.split(" ")[:3] for line in lines]
    lines = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines = pd.to_datetime(lines)
    return pd.DataFrame({"date2":lines})

def get_holiday_names(fpath):
    # holiday_names are holidays + around Black Fridays
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.strip().split(" ")[:4] for line in lines]
    lines_dt = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines_dt = pd.to_datetime(lines_dt)
    lines_hol = [line[3] for line in lines]
    return pd.DataFrame({"date2":lines_dt, "holiday_name":lines_hol})

def preprocess(_df, is_train):
    
    df = _df.copy()
    
    # date
    df['date2'] = pd.to_datetime(df['date'])
    
    # weekday
    df['weekday'] = df.date2.dt.weekday
    df['is_weekend'] = df.date2.dt.weekday.isin([5,6])
    df['is_holiday'] = df.date2.isin(holidays.date2)
    df['is_holiday_weekday'] = df.is_holiday & (df.is_weekend == False)
    df['is_holiday_weekend'] = df.is_holiday &  df.is_weekend

    # bool to int (maybe no meaning)
    df.is_weekend = np.where(df.is_weekend, 1, 0)
    df.is_holiday = np.where(df.is_holiday, 1, 0)
    df.is_holiday_weekday = np.where(df.is_holiday_weekday, 1, 0)
    df.is_holiday_weekend = np.where(df.is_holiday_weekend, 1, 0)
    
    # day, month, year
    df['day'] = df.date2.dt.day
    df['month'] = df.date2.dt.month
    df['year'] = df.date2.dt.year
    
    # around BlackFriday
    df = pd.merge(df, holiday_names, on='date2', how = 'left')
    df.loc[df.holiday_name.isnull(), "holiday_name"] = ""

    around_BlackFriday = ["BlackFridayM3", "BlackFridayM2", "ThanksgivingDay", "BlackFriday",
                          "BlackFriday1", "BlackFriday2", "BlackFriday3"]
    df["around_BlackFriday"] = np.where(df.holiday_name.isin(around_BlackFriday), 
                                        df.holiday_name, "Else")

    return df

In [14]:
holidays = get_holidays("walmart_holidays.txt")
holiday_names = get_holiday_names("walmart_holiday_names.txt")

In [15]:
df = preprocess(df, True)

In [16]:
# counting M

columns = list(df.columns)

def count_M(columns):
    ls_M_counting = []
    ls_M_columns= []
    ls_all_counting = []
    
    for i in columns:
        a = df[i].value_counts()
        b = df[i].name
        
        if 'M' in a:
            ls_M_counting.append(a['M'])
            ls_M_columns.append(b)
            ls_all_counting.append(a.sum())
    return ls_M_counting, ls_M_columns, ls_all_counting

        
    
k = count_M(columns)
k = np.array(k).T

M_columns = ['M', 'column', 'total']
dfdf = pd.DataFrame(data = k, columns = M_columns)
dfdf['M'] = dfdf['M'].astype(int)
dfdf['total'] = dfdf['total'].astype(int)
dfdf['Pct'] = dfdf['M'] / dfdf['total'] * 100
dfdf.sort_values(by='Pct', ascending=False)

M       column   total        Pct
3   79710       depart  118696  67.154748
8   42900     snowfall  118696  36.142751
11   4957     sealevel  118696   4.176215
2    4520         tavg  118696   3.808047
6    4520         heat  118696   3.808047
7    4520         cool  118696   3.808047
5    4492      wetbulb  118696   3.784458
0    3018         tmax  118696   2.542630
1    3016         tmin  118696   2.540945
10   2987  stnpressure  118696   2.516513
14   2759     avgspeed  118696   2.324425
9    2706  preciptotal  118696   2.279774
4    2571     dewpoint  118696   2.166038
12   2190  resultspeed  118696   1.845050
13   2190    resultdir  118696   1.845050

In [17]:
# M -> numeric

for col in df.columns:
    if df[col].name == 'weekday' or df[col].name == 'codesum' or df[col].name == 'holiday_name' or df[col].name == 'around_BlackFriday': continue
    elif df[col].dtypes == 'object':
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else: continue


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118696 entries, 0 to 118695
Data columns (total 68 columns):
date                  118696 non-null datetime64[ns]
store_nbr             118696 non-null int64
item_nbr              118696 non-null int64
units                 118696 non-null int64
station_nbr           118696 non-null int64
tmax                  115678 non-null float64
tmin                  115680 non-null float64
tavg                  114176 non-null float64
depart                38986 non-null float64
dewpoint              116125 non-null float64
wetbulb               114204 non-null float64
heat                  114176 non-null float64
cool                  114176 non-null float64
sunrise               49541 non-null float64
sunset                49541 non-null float64
codesum               118696 non-null object
snowfall              74553 non-null float64
preciptotal           101316 non-null float64
stnpressure           115709 non-null float64
sealevel             

In [20]:
df

date  store_nbr  item_nbr  units  station_nbr   tmax  tmin  tavg  \
0      2012-11-15         37         5   5568           13   44.0  24.0  34.0   
1      2012-11-15         25        45     70           13   44.0  24.0  34.0   
2      2012-11-15         25         9     70           13   44.0  24.0  34.0   
3      2012-11-15         15        45     62           13   44.0  24.0  34.0   
4      2012-11-15         32         9     52           13   44.0  24.0  34.0   
5      2012-11-15         40        45     51           13   44.0  24.0  34.0   
6      2012-11-15         37        45     44           13   44.0  24.0  34.0   
7      2012-11-15         32        45     36           13   44.0  24.0  34.0   
8      2012-11-15         15         5     29           13   44.0  24.0  34.0   
9      2012-11-15         40         5     29           13   44.0  24.0  34.0   
10     2012-11-15         25        81      8           13   44.0  24.0  34.0   
11     2012-11-15         15        93      5           13   44.0  24.0  34.0   
12     2012-11-15         15        15      4           13   44.0  24.0  34.0   
13     2012-11-15         40        15      2           13   44.0  24.0  34.0   
14     2012-11-15         25        12      1           13   44.0  24.0  34.0   
15     2012-11-15         15        17      1           13   44.0  24.0  34.0   
16     2012-11-15         32        19      1           13   44.0  24.0  34.0   
17     2012-11-15         37        15      1           13   44.0  24.0  34.0   
18     2013-11-21         37         5   3369           13   28.0  11.0  20.0   
19     2013-11-21         25        45     61           13   28.0  11.0  20.0   
20     2013-11-21         32         9     48           13   28.0  11.0  20.0   
21     2013-11-21         40        45     41           13   28.0  11.0  20.0   
22     2013-11-21         15        45     39           13   28.0  11.0  20.0   
23     2013-11-21         32        45     33           13   28.0  11.0  20.0   
24     2013-11-21         25         9     27           13   28.0  11.0  20.0   
25     2013-11-21         40         5     24           13   28.0  11.0  20.0   
26     2013-11-21         37        45     22           13   28.0  11.0  20.0   
27     2013-11-21         15         5     19           13   28.0  11.0  20.0   
28     2013-11-21         37        15     10           13   28.0  11.0  20.0   
29     2013-11-21         15        15      7           13   28.0  11.0  20.0   
...           ...        ...       ...    ...          ...    ...   ...   ...   
118666 2012-11-22         39        36      2            8   79.0  50.0   NaN   
118667 2012-11-22         39         9      1            8   79.0  50.0   NaN   
118668 2014-07-19         39        36      2            8   89.0  73.0  81.0   
118669 2014-07-19         39         9      2            8   89.0  73.0  81.0   
118670 2014-08-06         39         9      2            8   99.0  73.0  86.0   
118671 2014-08-16         39       111      2            8  102.0  79.0  91.0   
118672 2014-08-16         39        36      1            8  102.0  79.0  91.0   
118673 2014-03-28         39        36      2            8   85.0  60.0  73.0   
118674 2014-06-03         39        36      2            8   90.0  75.0  83.0   
118675 2014-06-03         39         9      2            8   90.0  75.0  83.0   
118676 2014-06-03         39       103      1            8   90.0  75.0  83.0   
118677 2012-03-20          1        28      2            1   82.0  41.0  62.0   
118678 2012-03-20          1         9      1            1   82.0  41.0  62.0   
118679 2012-01-07          1         9      2            1   57.0  28.0  43.0   
118680 2012-01-07          1        28      1            1   57.0  28.0  43.0   
118681 2014-02-18          1        28      2            1   30.0   6.0  18.0   
118682 2014-02-18          1        47      2            1   30.0   6.0  18.0   
118683 2014-02-18          1         9      

In [21]:
item_nbr = df.item_nbr.unique()
item_nbr

array([  5,  45,   9,  81,  93,  15,  12,  17,  19,  49,  50,  53,  38,
        51,  25,  16,  39,  85,  37,  71,  44,  41,   8,  92,  91,  48,
        56, 109,  18,  35,  13,  65,  52,  98,  31,  10,  72,  29,   6,
        27,  96,  90,  97,  76,  43, 107,  11,  82,  14, 101,  60,  68,
        20,  62,  21,  61,  58,  55,  86,  36,  23,  46,   7,  77,  64,
        94,  74,  32,  84, 105,  42,   2,  67, 108, 106,  95,  57,  28,
        40,  30, 104, 102,  59,   1,  80,  87,  54,  78, 100,  66,  24,
        79,  73,  89, 110,   3,  47,  26,  99,  33,  70,   4,  83,  69,
        75,  88,  22,  34,  63, 103, 111])

In [22]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn import ensemble

In [47]:
# Decision tree

test = df[df.item_nbr == 5]
num_attr = ['tmax', 'tmin', 'dewpoint', 'wetbulb', 'heat', 'cool', 'preciptotal', 'stnpressure',
            'sealevel', 'resultspeed', 'resultdir', 'avgspeed', 'units']

test = test[num_attr]

test = test.dropna()
n_df = pd.DataFrame(preprocessing.normalize(test))

valueArray = test.values

NewValueArray = n_df.values

X = NewValueArray[:, 0:13]
Y = valueArray[:, 0:13]

test_size = 0.1
seed = 5

X_train, X_validate, Y_train, Y_validate = train_test_split(X, Y, test_size=test_size, random_state=seed)

regr = DecisionTreeRegressor(max_depth=5, min_samples_split = 2)
regr.fit(X_train, Y_train)

y = regr.predict(X_validate)
pd.DataFrame(data=y)

0          1          2          3          4          5   \
0     65.118780  37.921348  33.769663  43.551364  13.223114   0.000000   
1     79.629297  54.469357  49.153961  57.239163   0.849028   3.152466   
2     89.646373  67.085492  62.873057  68.374352   0.124352  13.734456   
3     35.165854  14.773171  12.848780  21.409756  39.773171   0.000000   
4     65.118780  37.921348  33.769663  43.551364  13.223114   0.000000   
5     82.055959  57.540698  53.638081  60.461483   0.287064   5.336483   
6     65.118780  37.921348  33.769663  43.551364  13.223114   0.000000   
7     66.572340  39.788652  35.449645  45.042553  11.570922   0.000000   
8     82.055959  57.540698  53.638081  60.461483   0.287064   5.336483   
9     65.118780  37.921348  33.769663  43.551364  13.223114   0.000000   
10    92.337209  70.206395  65.447674  70.714390   0.000000  16.527616   
11    82.055959  57.540698  53.638081  60.461483   0.287064   5.336483   
12    35.165854  14.773171  12.848780  21.409756  39.773171   0.000000   
13    61.713841  35.374806  30.905132  40.884914  16.208398   0.000000   
14    82.055959  57.540698  53.638081  60.461483   0.287064   5.336483   
15    65.118780  37.921348  33.769663  43.551364  13.223114   0.000000   
16    42.846626  20.102249  16.846626  26.519427  33.284254   0.000000   
17    65.118780  37.921348  33.769663  43.551364  13.223114   0.000000   
18    92.337209  70.206395  65.447674  70.714390   0.000000  16.527616   
19    66.572340  39.788652  35.449645  45.042553  11.570922   0.000000   
20    61.713841  35.374806  30.905132  40.884914  16.208398   0.000000   
21    35.165854  14.773171  12.848780  21.409756  39.773171   0.000000   
22    42.846626  20.102249  16.846626  26.519427  33.284254   0.000000   
23    93.699869  71.659895  67.171691  72.140236   0.000000  17.917431   
24    82.055959  57.540698  53.638081  60.461483   0.287064   5.336483   
25    93.699869  71.659895  67.171691  72.140236   0.000000  17.917431   
26    66.572340  39.788652  35.449645  45.042553  11.570922   0.000000   
27    59.901288  34.927039  31.467811  40.660944  17.356223   0.000000   
28    92.337209  70.206395  65.447674  70.714390   0.000000  16.527616   
29    66.572340  39.788652  35.449645  45.042553  11.570922   0.000000   
...         ...        ...        ...        ...        ...        ...   
1400  51.270027  26.388839  22.275428  32.584158  25.909091   0.000000   
1401  95.267327  72.460396  67.173267  72.554455   0.000000  19.103960   
1402  80.834008  55.328745  50.455870  58.219433   0.548178   3.884211   
1403  66.572340  39.788652  35.449645  45.042553  11.570922   0.000000   
1404  82.055959  57.540698  53.638081  60.461483   0.287064   5.336483   
1405  89.646373  67.085492  62.873057  68.374352   0.124352  13.734456   
1406  66.572340  39.788652  35.449645  45.042553  11.570922   0.000000   
1407  51.270027  26.388839  22.275428  32.584158  25.909091   0.000000   
1408  65.118780  37.921348  33.769663  43.551364  13.223114   0.000000   
1409  80.834008  55.328745  50.455870  58.219433   0.548178   3.884211   
1410  93.699869  71.659895  67.171691  72.140236   0.000000  17.917431   
1411  51.270027  26.388839  22.275428  32.584158  25.909091   0.000000   
1412  66.572340  39.788652  35.449645  45.042553  11.570922   0.000000   
1413  95.267327  72.460396  67.173267  72.554455   0.000000  19.103960   
1414  93.699869  71.659895  67.171691  72.140236   0.000000  17.917431   
1415  80.834008  55.328745  50.455870  58.219433   0.548178   3.884211   
1416  79.629297  54.469357  49.153961  57.239163   0.849028   3.152466   
1417  93.699869  71.659895  67.171691  72.140236   0.000000  17.917431   
1418  86.456897  64.086207  60.129310  65.750000   0.465517  11.008621   
1419  82.055959  57.540698  53.638081  60.461483   0.287064   5.336483   
1420  93.699869  71.659895  67.171691  72.140236   0.000000  17.917431   
1421  92.337209  70.206395  65.447674  70.714390   0.000000  16.527616   
1422  65.118780  